# Testing Model

In [45]:
# Import Library
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [47]:
# Load Dataset
df = pd.read_csv("data/yt_spam_comment.csv")
df.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [48]:
# Fungsi prepare_json() untuk membuat payload JSON dari komentar
def prepare_json(comment):
    feature_spec = {
        "CONTENT": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(comment, "utf-8")]))
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [49]:
# Fungsi predictions() untuk membuat request prediksi komentar
def predictions(comment):
    json_data = prepare_json(comment)

    endpoint = "http://localhost:8501/v1/models/spam-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json().get("predictions")
    if prediction:
        confidence = prediction[0][0]
        result = "Spam Comment" if confidence > 0.6 else "Bukan Spam Comment"
    else:
        result = "Tidak ada label"

    print(f"{result} (Confidence: {confidence})")

In [51]:
# Mengambil data
comment = df['CONTENT'][15]
print(comment)

Hello! Do you like gaming, art videos, scientific experiments, tutorials,  lyrics videos, and much, much more of that? If you do please check out our  channel and subscribe to it, we've just started, but soon we hope we will  be able to cover all of our expectations... You can also check out what  we've got so far!﻿


In [52]:
# Prediksi Komentar
predictions(comment)

Spam Comment (Confidence: 0.999898493)
